In [514]:
!git add Parser.ipynb
!git commit -m "added unified approach for persons and organizations"
!git push

[master 5560542] added unified approach for persons and organizations
 1 file changed, 343 insertions(+), 72 deletions(-)
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (5/5), 3.20 KiB | 0 bytes/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/DmitrySerg/OpenData
   ff136af..5560542  master -> master


## Цель
- Собрать базу открытых данных с сайта Московской патриархии о персоналиях, организациях, богослужебных указаниях и храмах, структурировать их и загрузить в хаб открытых данных

## Задача
- проинвентаризировать данные на сайте Московской патриархии
- написать парсеры и выгрузить данные в форматах CSV и JSON
- загрузить данные в хаб открытых данных http://hubofdata.ru в удобном виде.
## Требования
- открытый исходный код в Github под свободной лицензией
- межплатформенный код (возможность запуска на Linux/Windows)
- предусмотреть перенос всех метаданных в CKAN в виде тегов и атрибутов
- переносить все данные в CKAN или на отдельный хостинг (например, в Github)
## Данные
- Персоналии - http://www.patriarchia.ru/db/persons/
- Организации - http://www.patriarchia.ru/db/organizations/
- Богослужебные указания - http://www.patriarchia.ru/bu/
- Храмы - http://map.patriarhia.ru/
## Пожелания
- реализация в виде программы командной строки или веб-приложения
- возможность запуска на MacOSX
- использовать скриптовые языки такие как Python, Perl, R и другие.

In [314]:
import requests      
import numpy as np   
import pandas as pd  
import time          
from bs4 import BeautifulSoup

import os

import string

from tqdm import tqdm_notebook

In [489]:
main_link = "http://www.patriarchia.ru"
link_persons = "http://www.patriarchia.ru/db/persons/"
link_organizations = "http://www.patriarchia.ru/db/organizations/"
link_guidance = "http://www.patriarchia.ru/bu/"
link_churches = "http://map.patriarhia.ru/"

file_path = os.getcwd() + "/data/{}"

In [517]:
class Parser:
    
    def __init__(self):
        pass
    
    def getSoup(self, link):
        """
            Returns bs4-formatted page
        """
        html = requests.get(link)
        if html.status_code == 200:
            soup = BeautifulSoup(html.content, 'lxml')
        else:
            soup = None
        return soup
    
    def getAllLinks(self, dataset_type, url=None):
        """
            Returns a list with names, links and affiliations for persons and organizations
            or dates for guidance

            dataset_type: string
                persons/organizations/guidance
                
            url: string
                url to parse data from
        """
        
        if dataset_type=='guidance':
            datelist = pd.date_range(
                pd.to_datetime(start_date), end=pd.to_datetime(end_date)
            ).tolist()
            datelist = [date.strftime("%Y-%m-%d") for date in datelist]
            return datelist

        links = getSoup(url)
        links = links.find('div', attrs={"class":"main"})
        links = links.findAll('div', attrs={'class':'news'})

        dfLinks = []
        for link in links:
            name = link.find('h4').text
            linkParsed = main_link + link.find('a')['href']
            affiliation = link.find('div').text
            dfLinks.append({"name":name, "link":linkParsed, "affiliation":affiliation})

        return dfLinks

    def getPageData(link):
        """
            Returns a dict with parsed field:value pairs from page

            link: string
                link to page
        """

        data = {}

        # use printing version of page 
        link = link.replace("text", "print")

        # first - get the page content
        pageContent = getSoup(link)

        # trying to find any images
        try:
            image_link = pageContent.find('div', attrs={'class':'news_img'})
            image_link = image_link.find('img')['src']
            data.update({'image_link':image_link})
        except:
            pass

        # now concentrate on text
        pageContent = pageContent.find('div', attrs={'class':'main'})
        pageContent = pageContent.findAll("dl")

        # second - find all field-value pairs on page
        for text in pageContent:        
            number_of_fields = len(text.findAll('dt'))*2-1
            if number_of_fields>0:
                parsedFields = []
                currentValue = text.find('dt')
                parsedFields.append(' '.join(currentValue.findAll(text=True)).strip().replace(u'\xa0', u' '))

                while number_of_fields>0:
                    currentValue = currentValue.nextSibling
                    if currentValue != '\n':
                        value = ' '.join(currentValue.findAll(text=True)).strip().replace(u'\xa0', u' ')
                        if value:
                            parsedFields.append(value)
                            number_of_fields -= 1

                text = dict(zip(parsedFields[::2], parsedFields[1::2]))
                data.update(text)

        return data
    
    def getGuidanceData(date):
        """
            Returns dict with name:text pair for particular day
        """
        link = link_guidance + date + '/print.html'
        pageContent = getSoup(link)
        pageContent = pageContent.find('div', attrs={'class':'main'})
        pageContent = pageContent.findAll('p')

        name = pageContent[0].getText().strip()
        pageContent = [paragraph.getText() for paragraph in pageContent[1:]]
        text = "\n".join(pageContent)
        data = {'date':date, 'name':name, 'text':text}

        return data
    
    def saveDataFrame(self, final, dataset_type):
        """
            Save final file into csv and json files
            
            final: list
                parsed data from pages
                
            dataset_type: string
                persons or organizations
        """
        
        df = pd.DataFrame(final)
        df.columns = df.columns.str.strip(':')
        
        if dataset_type=="persons":
            csv_path = file_path.format("persons.csv")
            json_path = file_path.format("persons.json")
            column_names = ['name','affiliation','Страна','Дата рождения', 
                           'Дата пострига','Дата хиротонии', 'Дата смерти',
                           'День ангела', 'Епархия', 'Место работы', 'Образование', 
                           'Биография', 'Награды', 'Научные труды, публикации', 
                           'E-mail', 'Web-сайт', 'link', 'image_link']
            df = df[column_names]
        elif dataset_type=="organizations":
            csv_path = file_path.format("organizations.csv")
            json_path = file_path.format("organizations.json")
        else:
            csv_path = file_path.format("guidance.csv")
            json_path = file_path.format("guidance.json")
        
        df.to_csv(csv_path, encoding='utf8')
        df.to_json(json_path)
            
            
            
    def runCrawler(self, links_data, dataset_type='other'):
        """
            Iterate over all links in links_data and return final dataset
            
            links_data: list
                list with links to pages
            
            dataset_type: string
                guidance or other
        """
        final = []

        for page in tqdm_notebook(links_data):
            try:
                if dataset_type=='guidance':
                    currentDateData = self.getGuidanceData(page)
                    final.append(currentDateData)
                else:
                    data = self.getPageData(page['link'])
                    page.update(data)
                    final.append(page)
                time.sleep(0.5)
            except Exception as e:
                # print(e)                           # DEBUG
                # print('problem',  person['link'])  # DEBUG
                time.sleep(30)
                
        return final
        
    
    def runParsing(self, dataset_type):
        
        if dataset_type=='persons':
            link = link_persons
        elif dataset_type=='organizations':
            link = link_organizations
        else:
            pass
        
        allLinks = self.getAllLinks(dataset_type, link)
        data = self.runCrawler(allLinks, dataset_type)
        self.saveDataFrame(data, dataset_type)

In [451]:
parser = Parser()
parser.runParsing('persons')

In [457]:
def getSoup(link):
    """
        Returns bs4-formatted page
    """
    html = requests.get(link)
    if html.status_code == 200:
        soup = BeautifulSoup(html.content, 'lxml')
    else:
        soup = None
    return soup

def getAllLinks(url):
    """
        Returns a list with names, links and affiliations    
        
        url: string
            url to parse data from
    """
    
    links = getSoup(url)
    links = links.find('div', attrs={"class":"main"})
    links = links.findAll('div', attrs={'class':'news'})
    
    dfLinks = []
    for link in links:
        name = link.find('h4').text
        linkParsed = main_link + link.find('a')['href']
        affiliation = link.find('div').text
        dfLinks.append({"name":name, "link":linkParsed, "affiliation":affiliation})
        
    return dfLinks

def getPageData(link):
    """
        Returns a dict with parsed field:value pairs from page
    
        link: string
            link to page
    """
    
    data = {}
    
    # use printing version of page 
    if 'text' in link:
        link = link.replace("text", "print")
    
    # first - get the page content
    pageContent = getSoup(link)
    
    # trying to find any images
    try:
        image_link = pageContent.find('div', attrs={'class':'news_img'})
        image_link = image_link.find('img')['src']
        data.update({'image_link':image_link})
    except:
        pass
    
    # now concentrate on text
    pageContent = pageContent.find('div', attrs={'class':'main'})
    pageContent = pageContent.findAll("dl")

    # second - find all field-value pairs on page
    for text in pageContent:        
        number_of_fields = len(text.findAll('dt'))*2-1
        if number_of_fields>0:
            parsedFields = []
            currentValue = text.find('dt')
            parsedFields.append(' '.join(currentValue.findAll(text=True)).strip().replace(u'\xa0', u' '))

            while number_of_fields>0:
                currentValue = currentValue.nextSibling
                if currentValue != '\n':
                    value = ' '.join(currentValue.findAll(text=True)).strip().replace(u'\xa0', u' ')
                    if value:
                        parsedFields.append(value)
                        number_of_fields -= 1

            text = dict(zip(parsedFields[::2], parsedFields[1::2]))
            data.update(text)
            
    return data

In [440]:
final = []
for page in tqdm_notebook(dfOrganizations):
    try:
        data = getPageData(page['link'])
        page.update(data)
        final.append(page)
        time.sleep(0.6)
    except Exception as e:
        # print(e)                           # DEBUG
        # print('problem',  person['link'])  # DEBUG
        time.sleep(30)

In [497]:
start_date = '2009-01-14'
end_date = '2019-01-13'

In [505]:
final = []
for date in tqdm_notebook(datelist):
    try:
        currentDateData = getGuidanceData(date)
        final.append(currentDateData)
        time.sleep(0.5)
    except Exception as e:
        print(e)                           # DEBUG
        print('problem',  date)  # DEBUG
        time.sleep(30)

'NoneType' object has no attribute 'find'
problem http://www.patriarchia.ru/db/text/80876.html
'NoneType' object has no attribute 'find'
problem http://www.patriarchia.ru/db/text/80876.html



In [506]:
df = pd.DataFrame(final)

In [513]:
df.head()

,name,text
0,Среда. Обре́зание Господне. Свт. Василия Велик...,Примечание. В Типиконе сказано: «А́ще храм свя...
1,Четверг. Предпразднство Богоявления. Прп. Сера...,"Свт. Сильве́стра, папы Римского. Прав. Иулиани..."
2,Пятница. Предпразднство Богоявления. Прор. Мал...,Служба прор. Малахии и мч. Гордия не имеет пра...
3,Суббота пред Богоявлением. Предпразднство Бого...,"Свт. Евста́фия, архиеп. Сербского.\nПрп. Ахи́л..."
4,"Неделя 31-я по Пятидесятнице, пред Богоявление...",Прп. Синклитики́и Александрийской.\nВоскресная...


In [508]:
df.to_csv('data/guidance.csv', encoding='utf8')
df.to_json('data/guidance.json')

In [510]:
def getGuidanceData(date):
    """
        Returns dict with name:text pair for particular day
    """
    link = link_guidance + date + '/print.html'
    pageContent = getSoup(link)
    pageContent = pageContent.find('div', attrs={'class':'main'})
    pageContent = pageContent.findAll('p')
    
    name = pageContent[0].getText().strip()
    pageContent = [paragraph.getText() for paragraph in pageContent[1:]]
    text = "\n".join(pageContent)
    data = {'date':date, 'name':name, 'text':text}
    
    return data